# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [1]:
import os
import sys
import urllib.request
import requests

news_data = []

client_id = "7Rw0IMw6pollWxMoWmep"
client_secret = "MA_4fslk3O" # 발급받은 client_secret
encText = urllib.parse.quote("파이썬")#url에 파이썬 글자 붙여줌
url = "https://openapi.naver.com/v1/search/news?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과


request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request) #접속 하게끔 해줌
rescode = response.getcode() #request를 response 코드로 전달 

#200 나오면 정상임
if(rescode==200):
#    response_body = response.read()
    result = requests.get(response.geturl(),
                          headers={"X-Naver-Client-Id":client_id,
                                   "X-Naver-Client-Secret":client_secret}
                         )
    news_data.append(result.json()) # 키 , 값 형태인 json 형태로 만들기
#    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [2]:
#print(news_data)
print(len(news_data))
#print(news_data[0])
#print(news_data[0]['items'])
#print(len(news_data[0]['items']))
#print(news_data[0]['items'][2])
#print(news_data[0]['items'][7]['link'])

1


In [3]:
naver_news_link = []

for page in news_data:
    #print(page)
    page_news_link = []
    
    for item in page['items']:
        #print(item)
        temp_link = item['link']
        #print(temp_link)
        if "naver" in temp_link:
            page_news_link.append(temp_link)
    
    naver_news_link.append(page_news_link)
        

# 사이트 확인하기에 편한 코드 구조.
for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=144&aid=0000749797
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=008&aid=0004617386
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002106950


In [4]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [5]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome('driver/chromedriver')

In [6]:
naver_news_title = []
naver_news_content = []


for n in tqdm_notebook(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

<ipython-input-6-326f55771db9>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(naver_news_link))):


  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-6-326f55771db9>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(naver_news_link[n]))):


  0%|          | 0/3 [00:00<?, ?it/s]

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=144&aid=0000749797
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=008&aid=0004617386
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002106950
['안양대, ‘코딩학 개론’ 프로그램 개설', "안양대 대학일자리센터, '코딩학개론' 프로그램 진행", '안랩, 디지털 직무 교육 사회공헌 프로그램 ‘안랩샘’ 12기 온라인 수료식 개최']
['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}   \t \t[스포츠경향]  안양대학교 대학일자리센터가 디지털 기초 이해도 확보 및 최신 트렌드에 맞는  IT 역량 강화를 위한 ‘코딩학개론’ 프로그램을 진행한다고  16 일 밝혔다.  10 월  11 일까지 3개월간 진행되는 이번 프로그램은 ‘코드잇’과 연계한 것으로,   IT  실무에서 많이 쓰이는 파이썬 기초, 머신 러닝, 웹퍼블리싱 등으로 교육과정을 구성하고, 학생들이 원하는 과목을 신청하여 자유롭게 수강하도록 개설된 것이 특징이다. 코드잇은 국내에서 유일하게 외주 제작 없이 대기업·아이비리그 출신 개발자들이 콘텐츠를 직접 기획, 제작해 유기적으로 연결된 강의를 제작하는 기업. 최근 글로벌 서비스를 개시해 국내뿐만 아니라 해외에도 높은 수준의 코딩 강의를 제공하고 있으며 높은 고객만족도와 향후 성장 가능성을 인정받아  2021 년 중소벤처기업부의 ‘아기 유니콘 기업’으로 선정되기도 했다.  이번 프로그램은 전공자, 비전공자 관계없이 코딩에 관심 있는 재학생 누구나 신청할 수 있으며 수강률  80 %이상, 기말평가  60 점 이상 달성

In [7]:
print(naver_news_title[0])

['안양대, ‘코딩학 개론’ 프로그램 개설', "안양대 대학일자리센터, '코딩학개론' 프로그램 진행", '안랩, 디지털 직무 교육 사회공헌 프로그램 ‘안랩샘’ 12기 온라인 수료식 개최']


In [8]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}   \t \t[스포츠경향]  안양대학교 대학일자리센터가 디지털 기초 이해도 확보 및 최신 트렌드에 맞는  IT 역량 강화를 위한 ‘코딩학개론’ 프로그램을 진행한다고  16 일 밝혔다.  10 월  11 일까지 3개월간 진행되는 이번 프로그램은 ‘코드잇’과 연계한 것으로,   IT  실무에서 많이 쓰이는 파이썬 기초, 머신 러닝, 웹퍼블리싱 등으로 교육과정을 구성하고, 학생들이 원하는 과목을 신청하여 자유롭게 수강하도록 개설된 것이 특징이다. 코드잇은 국내에서 유일하게 외주 제작 없이 대기업·아이비리그 출신 개발자들이 콘텐츠를 직접 기획, 제작해 유기적으로 연결된 강의를 제작하는 기업. 최근 글로벌 서비스를 개시해 국내뿐만 아니라 해외에도 높은 수준의 코딩 강의를 제공하고 있으며 높은 고객만족도와 향후 성장 가능성을 인정받아  2021 년 중소벤처기업부의 ‘아기 유니콘 기업’으로 선정되기도 했다.  이번 프로그램은 전공자, 비전공자 관계없이 코딩에 관심 있는 재학생 누구나 신청할 수 있으며 수강률  80 %이상, 기말평가  60 점 이상 달성 시 수료증이 발급된다. 김현태 대학일자리센터 팀장은 “4차 산업시대에 빠르게 변하는 트렌드에 맞춰 전공자들의 실질적인  IT 역량 강화와 비전공자들의 디지털 기초 이해도 확보가 기대된다”고 말했다. 앞서 안양대는 지난해 ‘대학일자리센터’ 개소식을 갖고 폭넓은 진로탐색지원, 맞춤형 진로 선택 지원, 구직활동 지원, 취업경쟁력 강화를 목표로 재학생 뿐만 아니라 지역 청년들의 진로와 취·창업을 위해 다양한 프로그램 운영에 박차를 가하고 있다.    // 본문 내용   ', '   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() 

In [9]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

3
3


In [11]:
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)